<a href="https://colab.research.google.com/github/ktichola/ML-and-DL-to-Gravitational-waves-analysis/blob/main/2_Kyriaki_Pulsar_Star_ML_Classification(ANN_99).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'kepler-labelled-time-series-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1074%2F1995%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240505%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240505T101239Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2c57424e59fc990242f6ba39ca879ee58fad3f683d98dcadf1fa9af0b22cec1c26965f85ed17d6ada8d7fc5922e8af45630febe0ef2047286d1b72fdf562c485cca9abf721d0896f51b3a25610588a8b33911302a7903160eac26eb144985d4dc9c7bfee02d62e1f2d80e119b17b753a99f204b09b57d2d961fa50e7856d3fb555bc02e8dba3829c13097187434f9282ea1fcc2d711222554cacb440500842001605a8100d7bf9dde9980f5845699e841c338c8f06deb86c02b79b4238070c31b267eade7988445f79e3df18cb9b03bf6edc00fd0f6a955419d3247a3ee2205ffa89ab6ffa1d37defbe14bc1088a86975d7d35ffc678499eecbbec1d30fb80a0,pulsar-star:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F110566%2F264393%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240505%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240505T101239Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9af7f4d57ccc0ca33587c496bd34d8cc25b05b117f8f70ae61491788461c9da8fcf1cc0452753619b08948ecf30d8b7a66aa04ac8aebd3728c056ab02e6469e18634bb2ef6a0751a12bf61acc9c63bcc75fbbda5458f23c213de45819912a20660d01667e70d67b7418e33e8e2523af8496bc5e631109b8ce705316136a5caa529354e3c2ba1d6915bb69fede230d3df89396b8bce2c9ae7bbbf846257c78bdde8012ae3a61741c37168d32a5a15c8b255de6095cf637488fd5a05eba2413ce13da0808646f3bc659a765636496feedb7a5ffa97ae58fe53431f9d26112ffca7c1b0176fd04865e601368fa15f42c026d97b1eaca04d1f7f18f5f7eba15f61ee,pulsar-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F644196%2F1142306%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240505%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240505T101239Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D45396dd9f86545628d50d134a8d8dda12158b9abf7921b719fe984dca6b5fbbcee2435b8327dd8333ccf12a2bccc610cbd1cba92ec150cd16991329bc03d213f7177ec89a72f5ef98561baca70088310fe461f4045611147e39b9280d5adbca7a808c725ded99a01ea4bf8c6d6156e83be0b0e4f380a7540d5d90debd25c1053e79709fe426f17856cf43a6578929038c3398662153cbe6eef62cffe057f2230ca8d20722149b0b9ef5539e5cef590903ac4a87e734a776b7a08e6505832bd72f7f7b532f7598fd061b85c4db943ea2994a17b899b255f309807cd117714324c67fc67a68e15382bdb798c2c1566c769c691e3c50f0fc4c0441162b4396df503,predicting-pulsar-starintermediate:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F845444%2F1442600%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240505%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240505T101239Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D84c2619a740b9b8f33d88dd9cee401cc910fb748a1adc675f400e4342dcf562205356b74f2f13393f5f5dbaf3250bbba8ccd4775950068890862f6502e4e6a9e108819d1d83555ae5feb32837debf348848bada916151a0b45fb39845e746cdf9015c5f4d3907299412d93a2727eaf29482f237d65f774e708a29d9503efb15149aba6d4494544747909ec34274d7198a8062b93700851d2328f8794e9ea0b138fe3cfec5d63e6ec8fd953ef637a8e4ecb18429f6afb67847c9bee8bcc7f3cf8453edeb2e5b712549cad27b1184caece5c1a2af480252016190863274008ab1cc62502cbb9c803170fd2e15a05ff0b950fa374b514bad5209c0048cc322b2238'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['pulsar-star', 'pulsar-stars-binary-classification-research', 'kepler-labelled-time-series-data', 'predicting-pulsar-starintermediate', 'pulsar-dataset']


# PRELIMINARY INFORMATION



## Exoplanet Hunting in Deep Space
**The Search for New Earths**

The data describe the change in flux (light intensity) of several thousand stars. Each star has a binary label of 2 or 1. 2 indicated that that the star is confirmed to have at least one exoplanet in orbit; some observations are in fact multi-planet systems.

As you can imagine, planets themselves do not emit light, but the stars that they orbit do. If said star is watched over several months or years, there may be a regular 'dimming' of the flux (the light intensity). This is evidence that there may be an orbiting body around the star; such a star could be considered to be a 'candidate' system. Further study of our candidate system, for example by a satellite that captures light at a different wavelength, could solidify the belief that the candidate can in fact be 'confirmed'.

Flux Diagram

In the above diagram, a star is orbited by a blue planet. At t = 1, the starlight intensity drops because it is partially obscured by the planet, given our position. The starlight rises back to its original value at t = 2. The graph in each box shows the measured flux (light intensity) at each time interval.

**Description**
Trainset:

* 5087 rows or observations.
* 3198 columns or features.
* Column 1 is the label vector. Columns 2 - 3198 are the flux values over time.
* 37 confirmed exoplanet-stars and 5050 non-exoplanet-stars.

Testset:

* 570 rows or observations.
* 3198 columns or features.
* Column 1 is the label vector. Columns 2 - 3198 are the flux values over time.
* 5 confirmed exoplanet-stars and 565 non-exoplanet-stars.

**Acknowledgements**
The data presented here are cleaned and are derived from observations made by the NASA Kepler space telescope. The Mission is ongoing - for instance data from Campaign 12 was released on 8th March 2017. Over 99% of this dataset originates from Campaign 3. To boost the number of exoplanet-stars in the dataset, confirmed exoplanets from other campaigns were also included.

To be clear, all observations from Campaign 3 are included. And in addition to this, confirmed exoplanet-stars from other campaigns are also included.

The datasets were prepared late-summer 2016.

Campaign 3 was used because 'it was felt' that this Campaign is unlikely to contain any undiscovered (i.e. wrongly labelled) exoplanets.

NASA open-sources the original Kepler Mission data and it is hosted at the Mikulski Archive. After being beamed down to Earth, NASA applies de-noising algorithms to remove artefacts generated by the telescope. The data - in the .fits format - is stored online. And with the help of a seasoned astrophysicist, anyone with an internet connection can embark on a search to find and retrieve the datafiles from the Archive.

The cover image is copyright © 2011 by Dan Lessmann

## PREDICTING A PULSAR STAR

*Dr Robert Lyon*

HTRU2 is a data set which describes a sample of pulsar candidates collected during the High Time Resolution Universe Survey .

Pulsars are a rare type of Neutron star that produce radio emission detectable here on Earth. They are of considerable scientific interest as probes of space-time, the inter-stellar medium, and states of matter .

As pulsars rotate, their emission beam sweeps across the sky, and when this crosses our line of sight, produces a detectable pattern of broadband radio emission. As pulsars rotate rapidly, this pattern repeats periodically. Thus pulsar search involves looking for periodic radio signals with large radio telescopes.

Each pulsar produces a slightly different emission pattern, which varies slightly with each rotation . Thus a potential signal detection known as a 'candidate', is averaged over many rotations of the pulsar, as determined by the length of an observation. In the absence of additional info, each candidate could potentially describe a real pulsar. However in practice almost all detections are caused by radio frequency interference (RFI) and noise, making legitimate signals hard to find.

Machine learning tools are now being used to automatically label pulsar candidates to facilitate rapid analysis. Classification systems in particular are being widely adopted, which treat the candidate data sets as binary classification problems. Here the legitimate pulsar examples are a minority positive class, and spurious examples the majority negative class.

The data set shared here contains 16,259 spurious examples caused by RFI/noise, and 1,639 real pulsar examples. These examples have all been checked by human annotators.

Each row lists the variables first, and the class label is the final entry. The class labels used are 0 (negative) and 1 (positive).

**Attribute Information:**
Each candidate is described by 8 continuous variables, and a single class variable. The first four are simple statistics obtained from the integrated pulse profile (folded profile). This is an array of continuous variables that describe a longitude-resolved version of the signal that has been averaged in both time and frequency . The remaining four variables are similarly obtained from the DM-SNR curve . These are summarised below:

* Mean of the integrated profile.
* Standard deviation of the integrated profile.
* Excess kurtosis of the integrated profile.
* Skewness of the integrated profile.
* Mean of the DM-SNR curve.
* Standard deviation of the DM-SNR curve.
* Excess kurtosis of the DM-SNR curve.
* Skewness of the DM-SNR curve.
* Class
* HTRU 2 Summary
* 17,898 total examples.
* 1,639 positive examples.
* 16,259 negative examples.

Source: https://archive.ics.uci.edu/ml/datasets/HTRU2

    Dr Robert Lyon
    University of Manchester
    School of Physics and Astronomy
    Alan Turing Building
    Manchester M13 9PL
    United Kingdom
    robert.lyon '@' manchester.ac.uk

### READING DATA

In [ ]:
train_data_exo = pd.read_csv("../input/kepler-labelled-time-series-data/exoTrain.csv")
test_data_exo = pd.read_csv("../input/kepler-labelled-time-series-data/exoTest.csv")
data = pd.read_csv("../input/pulsar-dataset/pulsar_stars.csv")
data.head()

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0


### DATA ANALYSIS

In [ ]:
data.corr()

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
Mean of the integrated profile,1.000000,0.547137,-0.873898,-0.738775,-0.298841,-0.307016,0.234331,0.144033,-0.673181
Standard deviation of the integrated profile,0.547137,1.000000,-0.521435,-0.539793,0.006869,-0.047632,0.029429,0.027691,-0.363708
Excess kurtosis of the integrated profile,-0.873898,-0.521435,1.000000,0.945729,0.414368,0.432880,-0.341209,-0.214491,0.791591
Skewness of the integrated profile,-0.738775,-0.539793,0.945729,1.000000,0.412056,0.415140,-0.328843,-0.204782,0.709528
Mean of the DM-SNR curve,-0.298841,0.006869,0.414368,0.412056,1.000000,0.796555,-0.615971,-0.354269,0.400876
Standard deviation of the DM-SNR curve,-0.307016,-0.047632,0.432880,0.415140,0.796555,1.000000,-0.809786,-0.575800,0.491535
Excess kurtosis of the DM-SNR curve,0.234331,0.029429,-0.341209,-0.328843,-0.615971,-0.809786,1.000000,0.923743,-0.390816
Skewness of the DM-SNR curve,0.144033,0.027691,-0.214491,-0.204782,-0.354269,-0.575800,0.923743,1.000000,-0.259117
target_class,-0.673181,-0.363708,0.791591,0.709528,0.400876,0.491535,-0.390816,-0.259117,1.000000


In [ ]:
train_data_exo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5087 entries, 0 to 5086
Columns: 3198 entries, LABEL to FLUX.3197
dtypes: float64(3197), int64(1)
memory usage: 124.1 MB


In [ ]:
test_data_exo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Columns: 3198 entries, LABEL to FLUX.3197
dtypes: float64(3197), int64(1)
memory usage: 13.9 MB


### SPLITING DATA

In [ ]:
X = data.drop(["target_class"],axis=1)
y = data["target_class"]

In [ ]:
x_train_exo = train_data_exo.drop(["LABEL"],axis=1)
y_train_exo = train_data_exo["LABEL"]
x_test_exo = test_data_exo.drop(["LABEL"],axis=1)
y_test_exo = test_data_exo["LABEL"]

In [ ]:
X_train,X_test, y_train , y_test = train_test_split(X,y,test_size=0.2)

### Creating Model and Predicting

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)
accuracy_lr=lr.score(X_test,y_test)
print("logistic regression accuracy:%{}".format(accuracy_lr*100))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


logistic regression accuracy:%97.98882681564245


In [ ]:
from tensorflow import set_random_seed
set_random_seed(101)
import tensorflow
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential # initialize neural network library
from keras.layers import Dense # build our layers library
def build_classifier():
    classifier = Sequential() # initialize neural network
    classifier.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'relu', input_dim = x_train_exo.shape[1]))
    classifier.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = x_train_exo, y = y_train_exo, cv = 3)
mean = accuracies.mean()
variance = accuracies.std()
print("Accuracy mean: "+ str(mean))
print("Accuracy variance: "+ str(variance))

Using TensorFlow backend.


Epoch 1/10
3391/3391 [==============================] - 1s 402us/step - loss: 0.2146 - acc: 1.0000
Epoch 2/10
3391/3391 [==============================] - 1s 219us/step - loss: 0.0174 - acc: 1.0000
Epoch 3/10
3391/3391 [==============================] - 1s 211us/step - loss: 0.0059 - acc: 1.0000
Epoch 4/10
3391/3391 [==============================] - 1s 218us/step - loss: 0.0034 - acc: 1.0000
Epoch 5/10
3391/3391 [==============================] - 1s 229us/step - loss: 0.0025 - acc: 1.0000
Epoch 6/10
3391/3391 [==============================] - 1s 198us/step - loss: 0.0022 - acc: 1.0000
Epoch 7/10
3391/3391 [==============================] - 1s 215us/step - loss: 0.0021 - acc: 1.0000
Epoch 8/10
3391/3391 [==============================] - 1s 209us/step - loss: 0.0020 - acc: 1.0000
Epoch 9/10
3391/3391 [==============================] - 1s 194us/step - loss: 0.0019 - acc: 1.0000
Epoch 10/10
1696/1696 [==============================] - 0s 122us/step
Epoch 1/10
3391/3391 [================

### ACCURACIES

In [ ]:
print("Predicting Pulsar mean: "+ str(mean))
print("Predicting Pulsar variance: "+ str(variance))

print("Exoplanets accuracy:%{}".format(accuracy_lr*100))

Predicting Pulsar mean: 0.9927279874213837
Predicting Pulsar variance: 0.010284178814426989
Exoplanets accuracy:%97.98882681564245


## Thanks for your reading. If you support me just vote up and leave a comment. Have a nice day.